# Projected 2021 Stats

In [1]:
#import dependencies
import pandas as pd
import requests
import time
import json
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect

# Website for 2021 Strength of Schedule
https://fftoolbox.fulltimefantasy.com/football/strength_of_schedule.cfm?type=i

In [2]:
query_url = f"https://api.sportsdata.io/v3/nfl/projections/json/PlayerGameProjectionStatsByWeek/2021REG/1?key=4339cb9aa27441699f1883848f498899"
response = requests.get(query_url).json()
print(json.dumps(response[0],indent=1))

{
 "GameKey": "202110116",
 "PlayerID": 18890,
 "SeasonType": 1,
 "Season": 2021,
 "GameDate": "2021-09-12T16:25:00",
 "Week": 1,
 "Team": "KC",
 "Opponent": "CLE",
 "HomeOrAway": "HOME",
 "Number": 15,
 "Name": "Patrick Mahomes",
 "Position": "QB",
 "PositionCategory": "OFF",
 "Activated": 1,
 "Played": 1,
 "Started": 1,
 "PassingAttempts": 19.0,
 "PassingCompletions": 13.6,
 "PassingYards": 165.2,
 "PassingCompletionPercentage": 37.8,
 "PassingYardsPerAttempt": 2.4,
 "PassingYardsPerCompletion": 3.3,
 "PassingTouchdowns": 0.7,
 "PassingInterceptions": 0.1,
 "PassingRating": 61.4,
 "PassingLong": 0.0,
 "PassingSacks": 0.0,
 "PassingSackYards": 0.0,
 "RushingAttempts": 1.1,
 "RushingYards": 5.2,
 "RushingYardsPerAttempt": 1.3,
 "RushingTouchdowns": 0.1,
 "RushingLong": 0.0,
 "ReceivingTargets": 0.0,
 "Receptions": 0.0,
 "ReceivingYards": 0.0,
 "ReceivingYardsPerReception": 0.0,
 "ReceivingTouchdowns": 0.0,
 "ReceivingLong": 0.0,
 "Fumbles": 0.0,
 "FumblesLost": 0.0,
 "PuntReturns": 0.0

In [3]:
wk1_2021_df = pd.DataFrame(columns=["Week","PlayerID", "Name","Team", "Position", "PassingYards", "Played", "PassingTouchdowns", "PassingInterceptions", "RushingTouchdowns", "ReceivingYards", "OpponentPositionRank","FantasyPoints"])
wk1_2021_df

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,OpponentPositionRank,FantasyPoints


In [5]:
for i in range(0,len(response)):
    currentItem = response[i]
    wk1_2021_df.loc[i] = [response[i]["Week"],response[i]["PlayerID"], 
                         response[i]["Name"],response[i]["Team"], response[i]["Position"], 
                         response[i]["PassingYards"], response[i]["Played"], response[i]["PassingTouchdowns"], 
                         response[i]["PassingInterceptions"], response[i]["RushingTouchdowns"], 
                         response[i]["ReceivingYards"], response[i]["OpponentPositionRank"], response[i]["FantasyPoints"]]
wk1_2021_df.head()

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,OpponentPositionRank,FantasyPoints
0,1,18890,Patrick Mahomes,KC,QB,165.2,1,0.7,0.1,0.1,0.0,13,12.6
1,1,20889,Kyler Murray,ARI,QB,139.4,1,0.5,0.2,0.1,0.0,15,12.5
2,1,19781,Lamar Jackson,BAL,QB,116.5,1,0.4,0.2,0.1,0.0,None,11.8
3,1,19801,Josh Allen,BUF,QB,142.4,1,0.5,0.2,0.1,0.0,None,11.7
4,1,14536,Russell Wilson,SEA,QB,149.6,1,0.6,0.2,0.0,0.0,3,11.5


In [6]:
print(wk1_2021_df["Team"].unique())

['KC' 'ARI' 'BAL' 'BUF' 'SEA' 'GB' 'TEN' 'DAL' 'LAC' 'PHI' 'TB' 'CIN'
 'CAR' 'MIN' 'LAR' 'ATL' 'WAS' 'CLE' 'SF' 'PIT' 'JAX' 'MIA' 'NO' 'LV'
 'NYG' 'NE' 'NYJ' 'DET' 'IND' 'DEN' 'CHI' 'HOU']


In [7]:
print(wk1_2021_df["Position"].unique())

['QB' 'RB' 'TE' 'WR' 'K']


In [8]:
print(len(wk1_2021_df))

892


In [9]:
STSchedule2021 = pd.read_csv('2021StrengthSchedule.csv')
STSchedule2021.head()

,TEAM,QB,RB,WR,TE,K,DEF,DL,LB,DB,PR,KR
0,NYJ,1,17,5,10,7,9,16,8,4,24,30
1,MIN,2,10,1,19,18,16,17,20,20,28,2
2,TEN,3,9,4,13,12,5,30,19,15,21,8
3,BAL,4,2,14,15,13,18,19,28,1,30,4
4,PHI,5,16,12,16,15,8,9,25,9,27,23


In [10]:
STSchedule2021.loc[(STSchedule2021.TEAM == 'LVR'),'TEAM']='LV'
STSchedule2021.head()

,TEAM,QB,RB,WR,TE,K,DEF,DL,LB,DB,PR,KR
0,NYJ,1,17,5,10,7,9,16,8,4,24,30
1,MIN,2,10,1,19,18,16,17,20,20,28,2
2,TEN,3,9,4,13,12,5,30,19,15,21,8
3,BAL,4,2,14,15,13,18,19,28,1,30,4
4,PHI,5,16,12,16,15,8,9,25,9,27,23


# QB

In [11]:
QBSTS = STSchedule2021.filter(items=['TEAM', 'QB'])
QBSTS.head()

,TEAM,QB
0,NYJ,1
1,MIN,2
2,TEN,3
3,BAL,4
4,PHI,5


In [12]:
QB2021_df = wk1_2021_df[wk1_2021_df['Position'] == "QB"]
QB2021_df

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,OpponentPositionRank,FantasyPoints
0,1,18890,Patrick Mahomes,KC,QB,165.2,1,0.7,0.1,0.1,0.0,13,12.6
1,1,20889,Kyler Murray,ARI,QB,139.4,1,0.5,0.2,0.1,0.0,15,12.5
2,1,19781,Lamar Jackson,BAL,QB,116.5,1,0.4,0.2,0.1,0.0,None,11.8
3,1,19801,Josh Allen,BUF,QB,142.4,1,0.5,0.2,0.1,0.0,None,11.7
4,1,14536,Russell Wilson,SEA,QB,149.6,1,0.6,0.2,0.0,0.0,3,11.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,1,22146,Ben DiNucci,DAL,QB,0.0,1,0.0,0.0,0.0,0.0,None,0.0
788,1,22160,Reid Sinnett,MIA,QB,0.0,0,0.0,0.0,0.0,0.0,None,0.0
818,1,22494,Kyle Trask,TB,QB,0.0,1,0.0,0.0,0.0,0.0,None,0.0
819,1,22497,Ian Book,NO,QB,0.0,1,0.0,0.0,0.0,0.0,1,0.0


In [13]:
QBSTSMerge = pd.merge(left=QB2021_df, right=QBSTS, how='left', left_on='Team', right_on='TEAM')
QBSTSMerge

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,OpponentPositionRank,FantasyPoints,TEAM,QB
0,1,18890,Patrick Mahomes,KC,QB,165.2,1,0.7,0.1,0.1,0.0,13,12.6,KC,16
1,1,20889,Kyler Murray,ARI,QB,139.4,1,0.5,0.2,0.1,0.0,15,12.5,ARI,17
2,1,19781,Lamar Jackson,BAL,QB,116.5,1,0.4,0.2,0.1,0.0,None,11.8,BAL,4
3,1,19801,Josh Allen,BUF,QB,142.4,1,0.5,0.2,0.1,0.0,None,11.7,BUF,29
4,1,14536,Russell Wilson,SEA,QB,149.6,1,0.6,0.2,0.0,0.0,3,11.5,SEA,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,1,22146,Ben DiNucci,DAL,QB,0.0,1,0.0,0.0,0.0,0.0,None,0.0,DAL,14
105,1,22160,Reid Sinnett,MIA,QB,0.0,0,0.0,0.0,0.0,0.0,None,0.0,MIA,25
106,1,22494,Kyle Trask,TB,QB,0.0,1,0.0,0.0,0.0,0.0,None,0.0,TB,20
107,1,22497,Ian Book,NO,QB,0.0,1,0.0,0.0,0.0,0.0,1,0.0,NO,30


In [14]:
dpdqbmerge = QBSTSMerge.drop(columns=['TEAM','OpponentPositionRank'],axis=1)
QBmerge2 = dpdqbmerge.rename(columns={"QB": "OpponentPositionRank"})
QBmerge2

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,FantasyPoints,OpponentPositionRank
0,1,18890,Patrick Mahomes,KC,QB,165.2,1,0.7,0.1,0.1,0.0,12.6,16
1,1,20889,Kyler Murray,ARI,QB,139.4,1,0.5,0.2,0.1,0.0,12.5,17
2,1,19781,Lamar Jackson,BAL,QB,116.5,1,0.4,0.2,0.1,0.0,11.8,4
3,1,19801,Josh Allen,BUF,QB,142.4,1,0.5,0.2,0.1,0.0,11.7,29
4,1,14536,Russell Wilson,SEA,QB,149.6,1,0.6,0.2,0.0,0.0,11.5,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,1,22146,Ben DiNucci,DAL,QB,0.0,1,0.0,0.0,0.0,0.0,0.0,14
105,1,22160,Reid Sinnett,MIA,QB,0.0,0,0.0,0.0,0.0,0.0,0.0,25
106,1,22494,Kyle Trask,TB,QB,0.0,1,0.0,0.0,0.0,0.0,0.0,20
107,1,22497,Ian Book,NO,QB,0.0,1,0.0,0.0,0.0,0.0,0.0,30


# RB

In [15]:
RB2021_df = wk1_2021_df[wk1_2021_df['Position'] == "RB"]
RB2021_df.head()

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,OpponentPositionRank,FantasyPoints
12,1,18877,Christian McCaffrey,CAR,RB,0.0,1,0.0,0.0,0.2,24.9,4,5.1
17,1,18872,Dalvin Cook,MIN,RB,0.0,1,0.0,0.0,0.2,15.1,13,4.8
20,1,17959,Derrick Henry,TEN,RB,0.0,1,0.0,0.0,0.2,2.5,None,4.6
25,1,18878,Alvin Kamara,NO,RB,0.0,1,0.0,0.0,0.2,22.8,14,4.5
33,1,19766,Saquon Barkley,NYG,RB,0.0,1,0.0,0.0,0.2,14.1,None,4.0


In [16]:
RBSTS = STSchedule2021.filter(items=['TEAM', 'RB'])
RBSTS.head()

,TEAM,RB
0,NYJ,17
1,MIN,10
2,TEN,9
3,BAL,2
4,PHI,16


In [17]:
RBSTSMerge = pd.merge(left=RB2021_df, right=RBSTS, how='left', left_on='Team', right_on='TEAM')
RBSTSMerge

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,OpponentPositionRank,FantasyPoints,TEAM,RB
0,1,18877,Christian McCaffrey,CAR,RB,0.0,1,0.0,0.0,0.2,24.9,4,5.1,CAR,12
1,1,18872,Dalvin Cook,MIN,RB,0.0,1,0.0,0.0,0.2,15.1,13,4.8,MIN,10
2,1,17959,Derrick Henry,TEN,RB,0.0,1,0.0,0.0,0.2,2.5,None,4.6,TEN,9
3,1,18878,Alvin Kamara,NO,RB,0.0,1,0.0,0.0,0.2,22.8,14,4.5,NO,14
4,1,19766,Saquon Barkley,NYG,RB,0.0,1,0.0,0.0,0.2,14.1,None,4.0,NYG,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,1,22955,B.J. Emmons,LV,RB,0.0,1,0.0,0.0,0.0,0.0,None,0.0,LV,30
203,1,22997,Tre Harbison,CLE,RB,0.0,0,0.0,0.0,0.0,0.0,None,0.0,CLE,1
204,1,23002,Nick Ralston,DAL,RB,0.0,0,0.0,0.0,0.0,0.0,None,0.0,DAL,20
205,1,23004,Adam Prentice,DEN,RB,0.0,0,0.0,0.0,0.0,0.0,None,0.0,DEN,11


In [18]:
dpdRBmerge = RBSTSMerge.drop(columns=['TEAM','OpponentPositionRank'],axis=1)
RBmerge2 = dpdRBmerge.rename(columns={"RB": "OpponentPositionRank"})
RBmerge2

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,FantasyPoints,OpponentPositionRank
0,1,18877,Christian McCaffrey,CAR,RB,0.0,1,0.0,0.0,0.2,24.9,5.1,12
1,1,18872,Dalvin Cook,MIN,RB,0.0,1,0.0,0.0,0.2,15.1,4.8,10
2,1,17959,Derrick Henry,TEN,RB,0.0,1,0.0,0.0,0.2,2.5,4.6,9
3,1,18878,Alvin Kamara,NO,RB,0.0,1,0.0,0.0,0.2,22.8,4.5,14
4,1,19766,Saquon Barkley,NYG,RB,0.0,1,0.0,0.0,0.2,14.1,4.0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,1,22955,B.J. Emmons,LV,RB,0.0,1,0.0,0.0,0.0,0.0,0.0,30
203,1,22997,Tre Harbison,CLE,RB,0.0,0,0.0,0.0,0.0,0.0,0.0,1
204,1,23002,Nick Ralston,DAL,RB,0.0,0,0.0,0.0,0.0,0.0,0.0,20
205,1,23004,Adam Prentice,DEN,RB,0.0,0,0.0,0.0,0.0,0.0,0.0,11


# WR

In [19]:
WR2021_df = wk1_2021_df[wk1_2021_df['Position'] == "WR"]
WR2021_df.head()

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,OpponentPositionRank,FantasyPoints
34,1,18082,Tyreek Hill,KC,WR,0.0,1,0.0,0.0,0.0,47.1,14,3.9
36,1,16470,Davante Adams,GB,WR,0.0,1,0.0,0.0,0.0,46.1,None,3.7
40,1,19802,Calvin Ridley,ATL,WR,0.0,1,0.0,0.0,0.0,50.6,None,3.7
45,1,16906,Stefon Diggs,BUF,WR,0.0,1,0.0,0.0,0.0,46.7,None,3.4
47,1,21042,A.J. Brown,TEN,WR,0.0,1,0.0,0.0,0.0,43.1,None,3.4


In [20]:
WRSTS = STSchedule2021.filter(items=['TEAM', 'WR'])
WRSTS.head()

,TEAM,WR
0,NYJ,5
1,MIN,1
2,TEN,4
3,BAL,14
4,PHI,12


In [21]:
WRSTSMerge = pd.merge(left=WR2021_df, right=WRSTS, how='left', left_on='Team', right_on='TEAM')
WRSTSMerge

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,OpponentPositionRank,FantasyPoints,TEAM,WR
0,1,18082,Tyreek Hill,KC,WR,0.0,1,0.0,0.0,0.0,47.1,14,3.9,KC,23
1,1,16470,Davante Adams,GB,WR,0.0,1,0.0,0.0,0.0,46.1,None,3.7,GB,20
2,1,19802,Calvin Ridley,ATL,WR,0.0,1,0.0,0.0,0.0,50.6,None,3.7,ATL,18
3,1,16906,Stefon Diggs,BUF,WR,0.0,1,0.0,0.0,0.0,46.7,None,3.4,BUF,26
4,1,21042,A.J. Brown,TEN,WR,0.0,1,0.0,0.0,0.0,43.1,None,3.4,TEN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,1,22991,Landen Akers,LAR,WR,0.0,0,0.0,0.0,0.0,0.0,None,0.0,LAR,8
338,1,23013,Damon Hazelton,GB,WR,0.0,0,0.0,0.0,0.0,0.0,None,0.0,GB,20
339,1,23090,C.J. Saunders,CAR,WR,0.0,0,0.0,0.0,0.0,0.0,10,0.0,CAR,3
340,1,23096,Travis Toivonen,SEA,WR,0.0,0,0.0,0.0,0.0,0.0,None,0.0,SEA,15


In [22]:
dpdWRmerge = WRSTSMerge.drop(columns=['TEAM','OpponentPositionRank'],axis=1)
WRmerge2 = dpdWRmerge.rename(columns={"WR": "OpponentPositionRank"})
WRmerge2

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,FantasyPoints,OpponentPositionRank
0,1,18082,Tyreek Hill,KC,WR,0.0,1,0.0,0.0,0.0,47.1,3.9,23
1,1,16470,Davante Adams,GB,WR,0.0,1,0.0,0.0,0.0,46.1,3.7,20
2,1,19802,Calvin Ridley,ATL,WR,0.0,1,0.0,0.0,0.0,50.6,3.7,18
3,1,16906,Stefon Diggs,BUF,WR,0.0,1,0.0,0.0,0.0,46.7,3.4,26
4,1,21042,A.J. Brown,TEN,WR,0.0,1,0.0,0.0,0.0,43.1,3.4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,1,22991,Landen Akers,LAR,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,8
338,1,23013,Damon Hazelton,GB,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,20
339,1,23090,C.J. Saunders,CAR,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,3
340,1,23096,Travis Toivonen,SEA,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,15


# TE

In [23]:
TE2021_df = wk1_2021_df[wk1_2021_df['Position'] == "TE"]
TE2021_df.head()

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,OpponentPositionRank,FantasyPoints
32,1,15048,Travis Kelce,KC,TE,0.0,1,0.0,0.0,0.0,50.8,15,4.0
62,1,19063,George Kittle,SF,TE,0.0,1,0.0,0.0,0.0,38.5,None,2.8
76,1,16964,Darren Waller,LV,TE,0.0,1,0.0,0.0,0.0,38.1,None,2.7
97,1,19803,Mark Andrews,BAL,TE,0.0,1,0.0,0.0,0.0,30.3,None,2.4
112,1,20805,T.J. Hockenson,DET,TE,0.0,1,0.0,0.0,0.0,30.1,0,2.2


In [24]:
TESTS = STSchedule2021.filter(items=['TEAM', 'TE'])
TESTS.head()

,TEAM,TE
0,NYJ,10
1,MIN,19
2,TEN,13
3,BAL,15
4,PHI,16


In [25]:
TESTSMerge = pd.merge(left=TE2021_df, right=TESTS, how='left', left_on='Team', right_on='TEAM')
TESTSMerge

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,OpponentPositionRank,FantasyPoints,TEAM,TE
0,1,15048,Travis Kelce,KC,TE,0.0,1,0.0,0.0,0.0,50.8,15,4.0,KC,4
1,1,19063,George Kittle,SF,TE,0.0,1,0.0,0.0,0.0,38.5,None,2.8,SF,27
2,1,16964,Darren Waller,LV,TE,0.0,1,0.0,0.0,0.0,38.1,None,2.7,LV,21
3,1,19803,Mark Andrews,BAL,TE,0.0,1,0.0,0.0,0.0,30.3,None,2.4,BAL,15
4,1,20805,T.J. Hockenson,DET,TE,0.0,1,0.0,0.0,0.0,30.1,0,2.2,DET,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,1,23050,John Raine,ATL,TE,0.0,0,0.0,0.0,0.0,0.0,None,0.0,ATL,3
189,1,23060,Jake Hausmann,NYG,TE,0.0,0,0.0,0.0,0.0,0.0,None,0.0,NYG,20
190,1,23063,Bernhard Seikovits,ARI,TE,0.0,0,0.0,0.0,0.0,0.0,5,0.0,ARI,30
191,1,23084,Shane Zylstra,MIN,TE,0.0,0,0.0,0.0,0.0,0.0,14,0.0,MIN,19


In [26]:
dpdTEmerge = TESTSMerge.drop(columns=['TEAM','OpponentPositionRank'],axis=1)
TEmerge2 = dpdTEmerge.rename(columns={"TE": "OpponentPositionRank"})
TEmerge2

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,FantasyPoints,OpponentPositionRank
0,1,15048,Travis Kelce,KC,TE,0.0,1,0.0,0.0,0.0,50.8,4.0,4
1,1,19063,George Kittle,SF,TE,0.0,1,0.0,0.0,0.0,38.5,2.8,27
2,1,16964,Darren Waller,LV,TE,0.0,1,0.0,0.0,0.0,38.1,2.7,21
3,1,19803,Mark Andrews,BAL,TE,0.0,1,0.0,0.0,0.0,30.3,2.4,15
4,1,20805,T.J. Hockenson,DET,TE,0.0,1,0.0,0.0,0.0,30.1,2.2,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,1,23050,John Raine,ATL,TE,0.0,0,0.0,0.0,0.0,0.0,0.0,3
189,1,23060,Jake Hausmann,NYG,TE,0.0,0,0.0,0.0,0.0,0.0,0.0,20
190,1,23063,Bernhard Seikovits,ARI,TE,0.0,0,0.0,0.0,0.0,0.0,0.0,30
191,1,23084,Shane Zylstra,MIN,TE,0.0,0,0.0,0.0,0.0,0.0,0.0,19


# Combine 2021 Position Data

In [27]:
Proj2021df = pd.concat([QBmerge2,RBmerge2,TEmerge2,WRmerge2])
Proj2021df

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,FantasyPoints,OpponentPositionRank
0,1,18890,Patrick Mahomes,KC,QB,165.2,1,0.7,0.1,0.1,0.0,12.6,16
1,1,20889,Kyler Murray,ARI,QB,139.4,1,0.5,0.2,0.1,0.0,12.5,17
2,1,19781,Lamar Jackson,BAL,QB,116.5,1,0.4,0.2,0.1,0.0,11.8,4
3,1,19801,Josh Allen,BUF,QB,142.4,1,0.5,0.2,0.1,0.0,11.7,29
4,1,14536,Russell Wilson,SEA,QB,149.6,1,0.6,0.2,0.0,0.0,11.5,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,1,22991,Landen Akers,LAR,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,8
338,1,23013,Damon Hazelton,GB,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,20
339,1,23090,C.J. Saunders,CAR,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,3
340,1,23096,Travis Toivonen,SEA,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,15


In [29]:
Proj2021df = Proj2021df.rename(columns={"FantasyPoints": "ProjFantasyPoints"})
Proj2021df

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,ProjFantasyPoints,OpponentPositionRank
0,1,18890,Patrick Mahomes,KC,QB,165.2,1,0.7,0.1,0.1,0.0,12.6,16
1,1,20889,Kyler Murray,ARI,QB,139.4,1,0.5,0.2,0.1,0.0,12.5,17
2,1,19781,Lamar Jackson,BAL,QB,116.5,1,0.4,0.2,0.1,0.0,11.8,4
3,1,19801,Josh Allen,BUF,QB,142.4,1,0.5,0.2,0.1,0.0,11.7,29
4,1,14536,Russell Wilson,SEA,QB,149.6,1,0.6,0.2,0.0,0.0,11.5,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,1,22991,Landen Akers,LAR,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,8
338,1,23013,Damon Hazelton,GB,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,20
339,1,23090,C.J. Saunders,CAR,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,3
340,1,23096,Travis Toivonen,SEA,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,15


In [30]:
Proj2021df = Proj2021df[Proj2021df['Position'].isin(['RB', 'QB', 'WR','TE'])]
Proj2021df

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,ProjFantasyPoints,OpponentPositionRank
0,1,18890,Patrick Mahomes,KC,QB,165.2,1,0.7,0.1,0.1,0.0,12.6,16
1,1,20889,Kyler Murray,ARI,QB,139.4,1,0.5,0.2,0.1,0.0,12.5,17
2,1,19781,Lamar Jackson,BAL,QB,116.5,1,0.4,0.2,0.1,0.0,11.8,4
3,1,19801,Josh Allen,BUF,QB,142.4,1,0.5,0.2,0.1,0.0,11.7,29
4,1,14536,Russell Wilson,SEA,QB,149.6,1,0.6,0.2,0.0,0.0,11.5,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,1,22991,Landen Akers,LAR,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,8
338,1,23013,Damon Hazelton,GB,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,20
339,1,23090,C.J. Saunders,CAR,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,3
340,1,23096,Travis Toivonen,SEA,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,15


In [32]:
Proj2021df['ActFantasyPoints'] = ''
Proj2021df

,Week,PlayerID,Name,Team,Position,PassingYards,Played,PassingTouchdowns,PassingInterceptions,RushingTouchdowns,ReceivingYards,ProjFantasyPoints,OpponentPositionRank,ActFantasyPoints
0,1,18890,Patrick Mahomes,KC,QB,165.2,1,0.7,0.1,0.1,0.0,12.6,16,
1,1,20889,Kyler Murray,ARI,QB,139.4,1,0.5,0.2,0.1,0.0,12.5,17,
2,1,19781,Lamar Jackson,BAL,QB,116.5,1,0.4,0.2,0.1,0.0,11.8,4,
3,1,19801,Josh Allen,BUF,QB,142.4,1,0.5,0.2,0.1,0.0,11.7,29,
4,1,14536,Russell Wilson,SEA,QB,149.6,1,0.6,0.2,0.0,0.0,11.5,24,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,1,22991,Landen Akers,LAR,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,8,
338,1,23013,Damon Hazelton,GB,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,20,
339,1,23090,C.J. Saunders,CAR,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,3,
340,1,23096,Travis Toivonen,SEA,WR,0.0,0,0.0,0.0,0.0,0.0,0.0,15,


In [33]:
Proj2021df.to_csv ('Projected2021.csv', index = False, header=True)